File to generate preprocess data to train U-Net model (create attention map & the combination files )

In [ ]:
import os #to joint the path
from glob import glob #used to extract images and masks path
from sklearn.model_selection import train_test_split #to split the dataset to train and validation
from tqdm import tqdm #The progress bar
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
!ls '/content/drive/MyDrive/Colab Notebooks/ADViTUNet/Dataset/ped1'

Frames	Frames_GT


In [ ]:
def create_dir(path):
  """ Create a directory """
  if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
create_dir("Data/Dataset/ped1")

In [ ]:
create_dir("Data/Preprocess/FramesAtt")
create_dir("Data/Preprocess/FramesJpg")
create_dir("Data/Preprocess/FramesComb")

In [ ]:
!cp -r '/content/drive/MyDrive/Colab Notebooks/ADViTUNet/Dataset/ped1/Frames/' '/content/Data/Dataset/ped1'
!cp -r '/content/drive/MyDrive/Colab Notebooks/ADViTUNet/Dataset/ped1/Frames_GT/' '/content/Data/Dataset/ped1'

In [ ]:
dataset_path = '/content/Data/Dataset/ped1'
frames = sorted(glob(f"{dataset_path}/*/*.tif"))
masks = sorted(glob(f"{dataset_path}/*/*.bmp"))

In [ ]:
%%capture
%pip install transformers
%pip install einops

In [ ]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
import cv2
from PIL import Image
import requests

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
pretrained_model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224', output_attentions=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:
import torch
from einops import reduce, rearrange
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def tif_to_jpg(images, save_path):
    for idx, x in tqdm(enumerate(images), total=len(images)):
      """Extracting the dir name and image name"""
      dir_name=x.split("/")[-3]
      name=x.split("/")[-1].split(".")[0]

      """Read the image and name"""
      xim=cv2.imread(x, cv2.IMREAD_COLOR)

      tmp_image_name = f"{name}.jpg"

      image_path = os.path.join(save_path, tmp_image_name)

      cv2.imwrite(image_path, xim)

In [ ]:
#Copy Frames jpg Folder to Drive
!cp -r '/content/Data/Preprocess/FramesJpg' '/content/drive/MyDrive/Colab Notebooks/ADViTUNet/Data/Preprocess'

In [ ]:
# My Attention Function
def get_attention_map(img, get_mask=False):
  inputs = feature_extractor(images=img, return_tensors="pt")
  outputs = pretrained_model(**inputs)
  logits = outputs.logits
  attentions = outputs.attentions

  att_mat = torch.stack(attentions).squeeze(1)
  att_mat = reduce(att_mat, 'b h len1 len2 -> b len1 len2', 'mean')

  residual_att = torch.eye(att_mat.size(1))
  aug_att_mat = att_mat + residual_att
  aug_att_mat = aug_att_mat / aug_att_mat.sum(dim=-1).unsqueeze(-1)

  # Recursively multiply the weight matrices
  joint_attentions = torch.zeros(aug_att_mat.size())
  joint_attentions[0] = aug_att_mat[0]

  for n in range(1, aug_att_mat.size(0)):
      joint_attentions[n] = torch.matmul(aug_att_mat[n], joint_attentions[n-1])

  # Attention from the output token to the input space.
  v = joint_attentions[-1]
  grid_size = int(np.sqrt(aug_att_mat.size(-1)))
  mask = v[0, 1:].reshape(grid_size, grid_size).detach().numpy()

  if get_mask:
    result = cv2.resize(mask / mask.max(), img.size)
  else:
    mask = cv2.resize(mask / mask.max(), img.size)[..., np.newaxis]
    result = (mask * img).astype("uint8")
  #
  return result


In [ ]:
def attMapToImg(mask,x,save_path):
  name=x.split("/")[-1].split(".")[0]
  tmp_att_name = f"{name}.jpg"

  att_path = os.path.join(save_path, tmp_att_name)
  fig, (ax1) = plt.subplots()
  ax1.axis(False)
  ax1.imshow(mask, alpha = 1,cmap='jet')
  fig.savefig(att_path,bbox_inches='tight',pad_inches=0)
  plt.close(fig)

In [ ]:
# Paths
PathFrames = '/content/Data/Dataset/ped1/Frames'
PathFramesJpg = '/content/Data/Preprocess/FramesJpg'
PathFramesAtt = '/content/Data/Preprocess/FramesAtt'
PathCombFrmAtt = '/content/Data/Preprocess/FramesComb'

In [1]:
# Convert tif file to jpg
frames = sorted(glob(f"{PathFrames}/*.tif"))
tif_to_jpg(frames,PathFramesJpg)

In [2]:
# Generate attention map

frames_jpg = sorted(glob(f"{PathFramesJpg}/*.jpg"))
for idx, x in tqdm(enumerate(frames_jpg), total=len(frames_jpg)):
  # if idx>180 :
  # if idx<2 :
    image = Image.open(x)
    att_map=get_attention_map(image, True)
    attMapToImg(att_map,x, PathFramesAtt)
  # break

In [ ]:
#Copy Attention map Folder to Drive
!cp -r '/content/Data/Preprocess/FramesAtt/' '/content/drive/MyDrive/Colab Notebooks/ADViTUNet/Data/Preprocess'

In [ ]:
def combFrmAtt(images, attentions, save_path):
    att_path = PathFramesAtt + '/'
    H = 1
    W = 1
    for idx, x in tqdm(enumerate(images), total=len(images)):
      """Extracting the dir name and image name"""
      dir_name=x.split("/")[-3]
      name=x.split("/")[-1].split(".")[0]

      print(dir_name)

      dir_name_att=x.split("/")

      x_att = att_path+name+".jpg"

      """Read the image and name"""
      x1=cv2.imread(x, cv2.IMREAD_COLOR)
      x2=cv2.imread(x_att, cv2.IMREAD_COLOR)
      h, w, c = x2.shape
      frm=cv2.resize(x1, (w,h))
      comb=cv2.add(frm,x2)
      tmp_image_name = f"{name}.jpg"
      image_path = os.path.join(save_path, tmp_image_name)
      cv2.imwrite(image_path, comb)
      break

In [3]:
frames = sorted(glob(f"{PathFramesJpg}/*.jpg"))
attentions = sorted(glob(f"{PathFramesAtt}/*.jpg"))

combFrmAtt(frames,attentions,PathCombFrmAtt)

In [ ]:
#Copy results folder to Drive
!cp -r '/content/Data/Preprocess/FramesComb' '/content/drive/MyDrive/Colab Notebooks/ADViTUNet/Data/Preprocess'

In [ ]:
!ls '/content/drive/MyDrive/Colab Notebooks/ADViTUNet/Data/Preprocess'

FramesAtt  FramesComb  FramesJpg
